In [2]:
# =========================================================
# ULTRA FINAL LAST SUBMISSION — FIXED LIGHTGBM VERSION
# =========================================================
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

SEED = 42
np.random.seed(SEED)

# =========================================================
# LOAD
# =========================================================
df = pd.read_csv("merged_libur_cuaca_ispu_ndvi.csv", sep=";")
df["tanggal"] = pd.to_datetime(df["tanggal"], dayfirst=True)
df = df.sort_values(["lokasi_clean", "tanggal"]).reset_index(drop=True)

df = df.drop(columns=["max","parameter_pencemar_kritis","time","id","stasiun"], errors="ignore")

LABEL_MAP = {"BAIK":0,"SEDANG":1,"TIDAK SEHAT":2}
df = df[df["kategori"].notna()].copy()
df["target"] = df["kategori"].map(LABEL_MAP).astype(int)

# =========================================================
# REINDEX HARIAN
# =========================================================
def reindex_daily(g):
    idx = pd.date_range(g["tanggal"].min(), g["tanggal"].max(), freq="D")
    g = g.set_index("tanggal").reindex(idx)
    g["lokasi_clean"] = g["lokasi_clean"].iloc[0]
    return g.reset_index().rename(columns={"index":"tanggal"})

df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)

# =========================================================
# IMPUTATION
# =========================================================
df["pm_duakomalima"] = df.groupby("lokasi_clean")["pm_duakomalima"].transform(lambda x: x.fillna(x.median()))
df = df[df["target"].notna()].copy()

# =========================================================
# TIME FEATURES
# =========================================================
df["month"] = df["tanggal"].dt.month
df["dayofyear"] = df["tanggal"].dt.dayofyear
df["dayofweek"] = df["tanggal"].dt.dayofweek

df["month_sin"] = np.sin(2*np.pi*df["month"]/12)
df["month_cos"] = np.cos(2*np.pi*df["month"]/12)
df["doy_sin"] = np.sin(2*np.pi*df["dayofyear"]/365)
df["doy_cos"] = np.cos(2*np.pi*df["dayofyear"]/365)
df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)

# =========================================================
# POLLUTANT FEATURES
# =========================================================
POLLUTANT_COLS = [
    "pm_sepuluh","pm_duakomalima","ozon",
    "nitrogen_dioksida","sulfur_dioksida","karbon_monoksida"
]

for col in POLLUTANT_COLS:
    for lag in [1,2,3,7,14]:
        df[f"{col}_lag{lag}"] = df.groupby("lokasi_clean")[col].shift(lag)

for col in POLLUTANT_COLS:
    df[f"{col}_roll7_mean"] = df.groupby("lokasi_clean")[col].rolling(7).mean().reset_index(0,drop=True)
    df[f"{col}_roll7_std"]  = df.groupby("lokasi_clean")[col].rolling(7).std().reset_index(0,drop=True)

df = df.dropna().reset_index(drop=True)

# =========================================================
# ENCODING + SPLIT
# =========================================================
le = LabelEncoder()
df["lokasi_enc"] = le.fit_transform(df["lokasi_clean"])

SPLIT_DATE = "2024-01-01"
train_df = df[df["tanggal"] < SPLIT_DATE]
val_df   = df[df["tanggal"] >= SPLIT_DATE]

FEATURES = (
    POLLUTANT_COLS +
    [c for c in df.columns if "lag" in c or "roll" in c] +
    ["month_sin","month_cos","doy_sin","doy_cos","dow_sin","dow_cos","lokasi_enc"]
)

X_train, y_train = train_df[FEATURES], train_df["target"]
X_val,   y_val   = val_df[FEATURES],   val_df["target"]

# =========================================================
# CLASS WEIGHT
# =========================================================
freq = y_train.value_counts(normalize=True).to_dict()
class_weight = {k: 1/(v+1e-6) for k,v in freq.items()}

# =========================================================
# MODEL 1 — LIGHTGBM (SKLEARN API)
# =========================================================
lgb_model = LGBMClassifier(
    objective="multiclass",
    num_class=3,
    learning_rate=0.03,
    num_leaves=127,
    n_estimators=3000,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight=class_weight,
    random_state=SEED
)

lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric="multi_logloss")

# =========================================================
# MODEL 2 — RANDOM FOREST
# =========================================================
rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=12,
    class_weight="balanced",
    random_state=SEED,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# =========================================================
# STACKING META
# =========================================================
train_stack = np.hstack([lgb_model.predict_proba(X_train), rf.predict_proba(X_train)])
val_stack   = np.hstack([lgb_model.predict_proba(X_val),   rf.predict_proba(X_val)])

meta = LogisticRegression(max_iter=2000, multi_class="multinomial")
meta.fit(train_stack, y_train)

val_prob = meta.predict_proba(val_stack)
print("VAL Macro-F1:", f1_score(y_val, val_prob.argmax(1), average="macro"))

# =========================================================
# THRESHOLD SEARCH
# =========================================================
best_t = np.array([0.4,0.35,0.25])
best_f1 = 0

for t0 in np.linspace(0.3,0.6,10):
    for t1 in np.linspace(0.25,0.5,10):
        for t2 in np.linspace(0.2,0.45,10):

            pred = val_prob.argmax(1)
            pred[val_prob[:,0] > t0] = 0
            pred[val_prob[:,1] > t1] = 1
            pred[val_prob[:,2] > t2] = 2

            f1 = f1_score(y_val, pred, average="macro")
            if f1 > best_f1:
                best_f1, best_t = f1, np.array([t0,t1,t2])

print("Best threshold:", best_t, "F1:", best_f1)

# =========================================================
# TEST PRED
# =========================================================
sub = pd.read_csv("sample_submission.csv")
sub["tanggal"] = pd.to_datetime(sub["id"].str.split("_").str[0])
sub["lokasi_clean"] = sub["id"].str.split("_").str[1]
sub["lokasi_enc"] = le.transform(sub["lokasi_clean"])

last_hist = df.sort_values("tanggal").groupby("lokasi_clean").tail(1)

rows = []
for _, r in sub.iterrows():
    hist = last_hist[last_hist["lokasi_clean"]==r["lokasi_clean"]].iloc[0].copy()

    hist["month_sin"] = np.sin(2*np.pi*r["tanggal"].month/12)
    hist["month_cos"] = np.cos(2*np.pi*r["tanggal"].month/12)
    hist["doy_sin"]   = np.sin(2*np.pi*r["tanggal"].dayofyear/365)
    hist["doy_cos"]   = np.cos(2*np.pi*r["tanggal"].dayofyear/365)
    hist["dow_sin"]   = np.sin(2*np.pi*r["tanggal"].dayofweek/7)
    hist["dow_cos"]   = np.cos(2*np.pi*r["tanggal"].dayofweek/7)

    rows.append(hist[FEATURES])

X_test = pd.DataFrame(rows)

test_stack = np.hstack([lgb_model.predict_proba(X_test), rf.predict_proba(X_test)])
test_prob = meta.predict_proba(test_stack)

pred = test_prob.argmax(1)
pred[test_prob[:,0] > best_t[0]] = 0
pred[test_prob[:,1] > best_t[1]] = 1
pred[test_prob[:,2] > best_t[2]] = 2

INV = {0:"BAIK",1:"SEDANG",2:"TIDAK SEHAT"}
sub["kategori"] = [INV[i] for i in pred]

sub.to_csv("ULTRA_FINAL_SUBMISSION.csv", index=False)
print("✅ ULTRA_FINAL_SUBMISSION.csv READY")


C:\Users\USER\AppData\Local\Temp\ipykernel_27692\2476968073.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("lokasi_clean", group_keys=False).apply(reindex_daily)
C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\joblib\externals\loky\backend\context.py", l

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7969
[LightGBM] [Info] Number of data points in the train set: 12160, number of used features: 55
[LightGBM] [Info] Start training from score -1.098614
[LightGBM] [Info] Start training from score -1.098609
[LightGBM] [Info] Start training from score -1.098613
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


VAL Macro-F1: 0.9002539307522609
Best threshold: [0.3  0.25 0.2 ] F1: 0.9024337024406086
✅ ULTRA_FINAL_SUBMISSION.csv READY
